In [ ]:
import os
import openai
import json
import textwrap
import builtins
import time
from openai import OpenAI
import inspect
import importlib.util
import ast
import shutil
import re
from typing import List
import instructor
from openai import Client
import sys
from typing import Optional
import subprocess
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [1]:
# This function allows you to see what the assistant is doing in the backed as it's being done
def wprint(*args, width=70, **kwargs):
    wrapper = textwrap.TextWrapper(width=width)
    wrapped_args = [wrapper.fill(str(arg)) for arg in args]
    builtins.print(*wrapped_args, **kwargs)

def is_input_string(input_value) -> bool:
    return isinstance(input_value, str)

# This function deals with all the communication with the Assistants API
def get_completion(message, agent, funcs, thread, client):
    # Create new message in the thread
    message_response = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=message
    )

    if is_input_string(agent):
        assistant_id = agent
    else:
        assistant_id = agent.id
    
    # Run the thread
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )

    while True:
        # Wait until run completes
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )

        if run.status in ['queued', 'in_progress']:
            time.sleep(1)
            continue

        if run.status == "requires_action":
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            tool_outputs = []
            for tool_call in tool_calls:
                print(f"Debug: Calling function {tool_call.function.name}", flush=True)

                wprint(f'\033[31mFunction: {tool_call.function.name}\033[0m')
                func = next((f for f in funcs if f.__name__ == tool_call.function.name), None)
                if func:
                    try:
                        # Assuming arguments are parsed correctly
                        func_instance = func(**eval(tool_call.function.arguments))  # Consider safer alternatives to eval
                        output = func_instance.run()

                        # Ensure output is a string
                        if not isinstance(output, str):
                            output = str(output)
                    except Exception as e:
                        output = f"Error: {e}"
                else:
                    output = "Function not found"
                wprint(f"\033[33m{tool_call.function.name}: {output}\033[0m")
                tool_outputs.append({"tool_call_id": tool_call.id, "output": output})

            run = client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs
            )
        elif run.status == "failed":
            raise Exception(f"Run Failed. Error: {run.last_error}")
        else:
            messages = client.beta.threads.messages.list(
                thread_id=thread.id
            )
            latest_message = messages.data[0].content[0].text.value
            return latest_message


In [ ]:
# This is a tool that can be given to chatgpt to use.
# This one specifically is for vetor search, but you can use this as a template to create your own tools.
# You just need the openai_schema, the __init__ to serve as an entry point for openai to give inputs, and a run function with all the code that should be run

class vector_search_tool:
    openai_schema = {
        "name": "vector_search_tool",
        "description": "Performs vector search on tool descriptions to find relevant tools based on a user query.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {"type": "string", "description": "User's search query."},
            },
            "required": ["query"]
        }
    }

    def __init__(self, query):
        self.query = query
        self.tool_database_path = 'tool_database.json'

    def run(self):
        try:
            # Load tools from JSON file
            with open(self.tool_database_path, 'r') as file:
                data = json.load(file)
                tool_data = data.get("tools", [])  # Extract the tools list

            # Ensure tool_data is a list
            if not isinstance(tool_data, list):
                raise ValueError("JSON data is not in expected list format")

            # Extract descriptions
            descriptions = [tool.get('description', '') for tool in tool_data]

            # Vectorize descriptions
            vectorizer = TfidfVectorizer()
            vectors = vectorizer.fit_transform(descriptions)

            # Vectorize query
            query_vec = vectorizer.transform([self.query])

            # Calculate cosine similarity
            cosine_similarities = cosine_similarity(query_vec, vectors).flatten()

            # Get top 5 relevant tools' code
            top_indices = cosine_similarities.argsort()[-5:][::-1]
            top_tool_codes = [tool_data[i]['code'] for i in top_indices]

            return top_tool_codes

        except Exception as e:
            return {"error": f"Error during search: {str(e)}"}

# # Example usage
# search_tool = tool_searcher(query="sphere volume")
# results = search_tool.run()
# print(results)


In [ ]:
# This code uses the tools you created previously and the get_completion function to run an assistant. 

client = openai.OpenAI(api_key="")

# Define any tools you want the assistant to use here
tool_searcher_tools = [vector_search_tool]

tool_searcher_agent = client.beta.assistants.create(
    name='Tool Searcher Agent',
    instructions="""
    Put your instructions for what you want the assistant to do here
    """,
    # Cheapest and fastest GPT model right now. 
    model="gpt-3.5-turbo-0125",
    # Formally give the assistant the tool here
    tools=[{"type": "function", "function": vector_search_tool.openai_schema},
           ]
)

# Create a new thread
thread = client.beta.threads.create()

# Main loop for user interaction
user_input = input("User: ")
message = get_completion(user_input, tool_searcher_agent, tool_searcher_tools, thread, client)
wprint(f"\033[34m{tool_searcher_agent.name}: {message}\033[0m")